In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import helper_functions
from azureml.core import Workspace, Dataset

In [2]:
subscription_id = '-----'
resource_group = '-----'
workspace_name = '------'

workspace = Workspace(subscription_id, resource_group, workspace_name)

In [3]:
def get_data(workspace, name):

    dataset = Dataset.get_by_name(workspace, name=name)
    return dataset.to_pandas_dataframe()

In [4]:
enrollment_fact = get_data(workspace, 'enrollment_fact')# RELACIÓN ENTRE CURSOS, PERIODOS Y ESTUDIANTES
course_dim = get_data(workspace, 'course_dim') # LISTA DE CURSOS EN GENERAL (ACADEMICOS Y EXTRA-ACADEMICOS)
user_dim = get_data(workspace, 'user_dim') # LISTA DE ESTUDIANTES
enrollment_term_dim = get_data(workspace, 'enrollment_term_dim')# LISTA DE PERIODOS 
pseudonym_dim = get_data(workspace, 'pseudonym_dim') # LISTA DE MATRICULAS
ecoa = get_data(workspace, 'GRUPO_ECOA') # CALIFICACIONES DE ECOA
alumno_df = get_data(workspace, 'DET_INT_ALUMNO') # RELACIÓN ESTUDIANTES CON ECOA Y MATERIAS
cursos_life = get_data(workspace, 'DWH_MATERIAS_EXTRA_ACADEMICAS') # LISTA DE MATERIAS EXTRA ACADEMICAS
materias_life = cursos_life.CLAVE_MATERIA.unique() # LISTA DE MATERIAS EXTRA ACADEMICAS

In [5]:
enrollment_fact = helper_functions.int_to_str(enrollment_fact)
course_dim = helper_functions.int_to_str(course_dim)
user_dim = helper_functions.int_to_str(user_dim)
enrollment_term_dim = helper_functions.int_to_str(enrollment_term_dim)
pseudonym_dim = helper_functions.int_to_str(pseudonym_dim)
alumno_df = helper_functions.int_to_str(alumno_df)
cursos_life = helper_functions.int_to_str(cursos_life)

In [6]:
# FILTRADO Y MODIFICACIÓN A COURSE_DIM

course_dim.code = course_dim.code.apply(lambda x : x[:x.find('.')]) # Obtener códigos generales de las materias
course_dim = course_dim[course_dim.code.isin(materias_life)] #HACER FILTRADO DE LAS MATERIAS QUE SOLO SON LIFE
course_dim.rename(columns={'id':'course_id'}, inplace = True)
course_dim = course_dim.loc[:, ['course_id','enrollment_term_id', 'name', 'code', 'start_at']] 

In [7]:
# FILTRADO Y MODIFICACIÓN A enrollment_fact

enrollment_fact = enrollment_fact.loc[:, ['enrollment_id', 'user_id', 'course_id', 'enrollment_term_id']]
enrollment_fact

,enrollment_id,user_id,course_id,enrollment_term_id
0,143280000000018596,103729482685559320,143280000000000874,143280000000000007
1,143280000000017210,369986378671339382,143280000000000874,143280000000000007
2,143280000000017212,571665427533582257,143280000000000874,143280000000000007
3,143280000000017216,-532628700693452303,143280000000000874,143280000000000007
4,143280000000017218,-516765647756560062,143280000000000874,143280000000000007
...,...,...,...,...
4627970,143280000003710540,-388274441878071802,143280000000204142,143280000000000141
4627971,143280000003592898,311511555886490383,143280000000204342,143280000000000141
4627972,143280000003598117,41939860565794830,143280000000204342,143280000000000141
4627973,143280000003665067,-235574164982587329,143280000000204342,143280000000000141


In [8]:
# FILTRADO Y MODIFICACIÓN A user_dim

user_dim.rename(columns={'id':'user_id', 'name':'user_name'}, inplace = True)
user_dim = user_dim.loc[:, ['user_id', 'user_name', 'time_zone']]

In [9]:
# FILTRADO Y MODIFICACIÓN A user_dim

pseudonym_dim.rename(columns={'unique_name':'matricula'}, inplace=True)
pseudonym_dim = pseudonym_dim.loc[:, ['id', 'user_id', 'matricula']]

In [10]:

tabla = pd.merge(enrollment_fact, course_dim, left_on=['course_id', 'enrollment_term_id' ], right_on=['course_id', 'enrollment_term_id'])
tabla2 = pd.merge(tabla, user_dim, left_on=['user_id'], right_on=['user_id'])
tabla3 = pd.merge(tabla2, pseudonym_dim, left_on=['user_id'], right_on=['user_id'])
tabla3['start_at'] = tabla3['start_at'].apply(helper_functions.ddate)

In [11]:
ecoa.CLAVE_EJER_ACAD_GRUPO = ecoa.CLAVE_EJER_ACAD_GRUPO.astype('str')

In [12]:
final = pd.DataFrame(columns=['matricula', 'course_code', 'course_name', 'ratings'])

for matricula in tqdm(tabla3.matricula.unique()[:100]):

    df1 = tabla3[tabla3.matricula == matricula].loc[:, ['matricula', 'user_name', 'start_at', 'code']]
    df2 = df1.merge(cursos_life, left_on=['code', 'start_at'], right_on=['CLAVE_MATERIA', 'CLAVE_EJERCICIO_ACADEMICO'])
    df3 = df2.merge(alumno_df, left_on = ['CLAVE_CAMPUS', 'CLAVE_EJERCICIO_ACADEMICO', 'matricula'], 
                            right_on = ['CLAVE_CAMPUS', 'CLAVE_EJERCICIO_ACADEMICO', 'MATRICULA'])

    df4 = df3.merge(ecoa, left_on = ['CLAVE_CAMPUS', 'CLAVE_EJERCICIO_ACADEMICO', 'CLAVE_MATERIA'],
                            right_on = ['CLAVE_CAMPUS_GRUPO', 'CLAVE_EJER_ACAD_GRUPO', 'CLAVE_MATERIA'])

    for c in df3.code.unique():
        p = helper_functions.promedio_p(df4[(df4['code']==c)]['EVAL'])
        df3.loc[(df3['code']==c), 'EVAL'] = int(p)

    df3.rename(columns={'EVAL':'ratings'}, inplace= True)
    final = pd.concat([final, df3])

final = final.drop_duplicates(subset= ['matricula', 'code', 'course_name']).reset_index()
final = final.loc[:, ['matricula', 'user_name', 'code', 'NOMBRE_MATERIA_CORTO', 'CLAVE_EJERCICIO_ACADEMICO', 'ratings']].copy()

100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


In [13]:
datastore = workspace.get_default_datastore()
# Uploading Pandas dataframe and registering it as a dataset
dataset = Dataset.Tabular.register_pandas_dataframe(
    final, datastore, "final_dataframe", show_progress=True
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/a44f93f7-30b4-4181-a4ad-b4fe879eed51/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
